# HuggingFace 模型

## MiniCPM-2B

- 安装`transformers>=4.36.0`以及`accelerate`后，运行以下代码

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
torch.manual_seed(0)

path = 'openbmb/MiniCPM-2B-dpo-bf16'
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForCausalLM.from_pretrained(path, torch_dtype=torch.bfloat16, device_map='cuda', trust_remote_code=True)

responds, history = model.chat(tokenizer, "四川省最高的山是哪座山, 它比K2高还是矮？差距多少？", temperature=0.5, top_p=0.8, repetition_penalty=1.02)
print(responds)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


四川省最高的山是贡嘎山，海拔高度为7556米。而K2峰（K2峰是K2峰峰顶的正式名称）是世界上第二高的山峰，海拔高度为8611米。

因此，贡嘎山的海拔高度比K2峰低。具体来说，贡嘎山的海拔高度比K2峰低1045米。


## MiniCPM-V

多模态（视觉）模型

需要安装flash-attn: `pip install flash_attn`

- 注意torch__version__ = 2.1.2+cu121是不支持的
```
$ pip install flash_attn
Collecting flash_attn
  Downloading flash_attn-2.5.2.tar.gz (2.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      fatal: not a git repository (or any of the parent directories): .git
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-vwvbptyl/flash-attn_4ab0bfafef2d42fdbe28a091588f3008/setup.py", line 112, in <module>
          _, bare_metal_version = get_cuda_bare_metal_version(CUDA_HOME)
        File "/tmp/pip-install-vwvbptyl/flash-attn_4ab0bfafef2d42fdbe28a091588f3008/setup.py", line 65, in get_cuda_bare_metal_version
          raw_output = subprocess.check_output([cuda_dir + "/bin/nvcc", "-V"], universal_newlines=True)
        File "/opt/anaconda3/envs/learning-in-general/lib/python3.10/subprocess.py", line 421, in check_output
          return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
        File "/opt/anaconda3/envs/learning-in-general/lib/python3.10/subprocess.py", line 503, in run
          with Popen(*popenargs, **kwargs) as process:
        File "/opt/anaconda3/envs/learning-in-general/lib/python3.10/subprocess.py", line 971, in __init__
          self._execute_child(args, executable, preexec_fn, close_fds,
        File "/opt/anaconda3/envs/learning-in-general/lib/python3.10/subprocess.py", line 1863, in _execute_child
          raise child_exception_type(errno_num, err_msg, err_filename)
      FileNotFoundError: [Errno 2] No such file or directory: '/opt/anaconda3/envs/learning-in-general/bin/nvcc'
      
      
      torch.__version__  = 2.1.2+cu121
      
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.
```

In [ ]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer
import os

model = AutoModel.from_pretrained('openbmb/MiniCPM-V', trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V', trust_remote_code=True)
model.eval().cuda()

# os expand path
image = Image.open(os.path.expanduser('~/Pictures/SD/SFW/girl-on-top-of-mountain.png')).convert('RGB')
question = 'What is in the image?'
msgs = [{'role': 'user', 'content': question}]

res, context, _ = model.chat(
    image=image,
    msgs=msgs,
    context=None,
    tokenizer=tokenizer,
    sampling=True,
    temperature=0.7
)
print(res)